In [104]:
# Bloque de importación de bibliotecas para realizar el modelo predictivo

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn.metrics import accuracy_score
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split




In [105]:
# Lectura del dataset obtenido desde Kraggle

df = pd.read_csv('diabetes_prediction_dataset.csv')

In [106]:
# Revisar si el dataset tiene datos "null" y dropear filas duplicadas

df.isnull().sum()
df.drop_duplicates(inplace = True)


In [107]:
# De acá se observa que algunas variables del dataset no son numéricas, como: "gender" y "smoking_history" por lo que deben ser encodeadas para realizar la predicción
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96146 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               96146 non-null  object 
 1   age                  96146 non-null  float64
 2   hypertension         96146 non-null  int64  
 3   heart_disease        96146 non-null  int64  
 4   smoking_history      96146 non-null  object 
 5   bmi                  96146 non-null  float64
 6   HbA1c_level          96146 non-null  float64
 7   blood_glucose_level  96146 non-null  int64  
 8   diabetes             96146 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 7.3+ MB


In [108]:
# Se observan los valores de las dos variables en cuestión para realizar el mapeo correspondiente a los niveles observados
df['gender'].value_counts()

Female    56161
Male      39967
Other        18
Name: gender, dtype: int64

In [109]:
df['smoking_history'].value_counts()

never          34398
No Info        32887
former          9299
current         9197
not current     6367
ever            3998
Name: smoking_history, dtype: int64

In [110]:
# Se realiza el mapeo y encode de las dos variables a transformar

labelEncode = pp.LabelEncoder()

genderMap = {'Female': 0, 'Male': 1, 'Other': 2}
df['gender'] = df['gender'].map(genderMap)

smokingHistoryMap = {'never': 0, 'No Info': 1, 'current': 2, 'former': 3, 'ever': 4, 'not current': 5}
df['smoking_history'] = df['smoking_history'].map(smokingHistoryMap)

In [111]:
# Se observan las métricas descriptivas para el dataset, además se ve que el valor mínimo para la edad es 0.08, por lo cuál es un valor "inválido" en este caso para la edad
df.describe()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
count,96146.000000,96146.000000,96146.000000,96146.000000,96146.000000,96146.000000,96146.000000,96146.000000,96146.000000
mean,0.416065,41.794326,0.077601,0.040803,1.320960,27.321461,5.532609,138.218231,0.088220
std,0.493287,22.462948,0.267544,0.197833,1.473466,6.767716,1.073232,40.909771,0.283616
min,0.000000,0.080000,0.000000,0.000000,0.000000,10.010000,3.500000,80.000000,0.000000
25%,0.000000,24.000000,0.000000,0.000000,0.000000,23.400000,4.800000,100.000000,0.000000
50%,0.000000,43.000000,0.000000,0.000000,1.000000,27.320000,5.800000,140.000000,0.000000
75%,1.000000,59.000000,0.000000,0.000000,2.000000,29.860000,6.200000,159.000000,0.000000
max,2.000000,80.000000,1.000000,1.000000,5.000000,95.690000,9.000000,300.000000,1.000000


In [112]:
# Se remueven los valores que no tienen sentido en este caso de la edad
df = df[df['age'].mod(1) == 0]
# Luego se puede volver a ejecutar la celda anterior para ver que efectivamente se removieron los valores no válidos

In [113]:
# Teniendo los datos ya preparados, se splitean los datos en los conjuntos para comenzar a entrenar los modelos predictivos

# Van todas las variables para predecir si se tiene diabetes o no
x = df.iloc[:,:-1].values

# Target
y = df.iloc[:,-1].values

# Se separan los datos en 80% para entrenamiento y 20% para evaluación
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

# Se realiza el featuring scaling para normalizar los datos
norm = pp.StandardScaler()
x_train = norm.fit_transform(x_train)
x_test = norm.transform(x_test)

In [114]:
# Se revisan algunos métodos predictivos
# KNeighborsClassifier (KNN)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9594199819408297


In [115]:
# SVM Lineal (Linear SVM)
from sklearn.svm import SVC
svm = SVC(kernel = 'linear')
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9579858713549689


In [116]:
# Árbol de clasificación (DecisionTreeClassifier)
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)
y_pred = tree.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9516120465289213


In [117]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
forest.fit(x_train, y_train)
y_pred = forest.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9672810325596218


In [118]:
# ANN (Artificial Neural Network)
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) 
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(x_train, y_train, batch_size=32, epochs = 100)
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(accuracy_score(y_test, y_pred))

Epoch 1/100
2354/2354 [==============================] - 2s 658us/step - loss: 0.1587 - accuracy: 0.9465
Epoch 2/100
2354/2354 [==============================] - 2s 678us/step - loss: 0.1233 - accuracy: 0.9573
Epoch 3/100
2354/2354 [==============================] - 2s 687us/step - loss: 0.1217 - accuracy: 0.9579
Epoch 4/100
2354/2354 [==============================] - 2s 723us/step - loss: 0.1211 - accuracy: 0.9578
Epoch 5/100
2354/2354 [==============================] - 2s 701us/step - loss: 0.1209 - accuracy: 0.9577
Epoch 6/100
2354/2354 [==============================] - 2s 683us/step - loss: 0.1205 - accuracy: 0.9577
Epoch 7/100
2354/2354 [==============================] - 2s 655us/step - loss: 0.1203 - accuracy: 0.9577
Epoch 8/100
2354/2354 [==============================] - 2s 638us/step - loss: 0.1201 - accuracy: 0.9580
Epoch 9/100
2354/2354 [==============================] - 1s 633us/step - loss: 0.1197 - accuracy: 0.9580
Epoch 10/100
2354/2354 [==============================]

In [124]:
# Ejemplo de predicción de una mujer de 37 años con hipertensión, sin enfermedades cardiacas, fuma siempre, 6.4 de hemoglobina y 100 de glucosa
ejemplo = norm.transform([[1,37.0, 1,0,4,24,6.8, 94]])
prediccion = ann.predict(ejemplo)
prediccion = (prediccion > 0.5)
print(prediccion)


1/1 [==============================] - 0s 13ms/step
[[False]]
